<a href="https://colab.research.google.com/github/Ino54/MA_GreenAI-Practical-Experiments/blob/main/bloom_pruning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ---------- Requirements ----------
%%writefile requirements.txt
transformers
accelerate
bitsandbytes
datasets
evaluate
sacrebleu
codecarbon>=2.5,<3
pynvml>=11.5.0
psutil
numpy
pandas
huggingface_hub

Writing requirements.txt


In [ ]:
!pip -q install -r requirements.txt
# häufige Konflikte still entfernen
!pip uninstall -y -q google-genai firebase-admin || true

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 43.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 517.6/517.6 kB 39.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 101.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 kB 8.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
firebase-admin 6.9.0 requires httpx[http2]==0.28.1, but you have httpx 0.27.2 which is incompatible.
google-genai 1.38.0 requires httpx<1.0.0,>=0.28.1, but you have httpx 0.27.2 which is incomp

In [ ]:
# ---------- Hugging Face Login via Colab-Secret ----------
from google.colab import userdata
from huggingface_hub import login
hf_token = userdata.get("HF_TOKEN")
if hf_token:
    login(hf_token); print("Hugging Face Login erfolgreich!")
else:
    print("WARNUNG: Kein HF_TOKEN gefunden")

Hugging Face Login erfolgreich!


In [ ]:
# ---------- Drive mounten & Zielordner prüfen ----------
import os, shutil, time, pathlib, platform, gc, re, math, warnings, inspect
warnings.filterwarnings("ignore")

from google.colab import drive
MOUNTPOINT = "/content/drive"
already = os.path.isdir(os.path.join(MOUNTPOINT, "MyDrive"))
if not already and os.path.isdir(MOUNTPOINT) and os.listdir(MOUNTPOINT):
    backup = f"/content/drive_stale_{int(time.time())}"
    shutil.move(MOUNTPOINT, backup)
    os.makedirs(MOUNTPOINT, exist_ok=True)
drive.mount(MOUNTPOINT, force_remount=(not already))

work_dir = "/content/drive/MyDrive/LLM-Effizienz/4_3_Effizienzstrategien/bloom_pruning"
if not os.path.isdir(work_dir):
    raise FileNotFoundError(
        f"Zielordner fehlt: {work_dir}\n"
        "Bitte diesen Ordner manuell in Google Drive anlegen und das Notebook erneut starten."
    )
os.chdir(work_dir)
project_dir = work_dir
print("Arbeitsordner:", os.getcwd())

Mounted at /content/drive
Arbeitsordner: /content/drive/MyDrive/LLM-Effizienz/4_3_Effizienzstrategien/bloom_pruning


In [ ]:
# ---------- Imports & Device ----------
import numpy as np, pandas as pd
import torch, psutil
from contextlib import nullcontext
from types import SimpleNamespace
from datasets import load_dataset
import evaluate

from transformers import AutoTokenizer, AutoModelForCausalLM, set_seed, BitsAndBytesConfig
from codecarbon import EmissionsTracker
import torch.nn.utils.prune as prune
import torch.nn as nn

# VRAM-Fragmentation entschärfen
import os as _os
_os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

set_seed(42)
device = "cuda" if torch.cuda.is_available() else "cpu"
if device == "cuda":
    gpu_name = torch.cuda.get_device_name(0)
    vram_total_gb = torch.cuda.get_device_properties(0).total_memory/(1024**3)
    torch.backends.cuda.matmul.allow_tf32 = True
else:
    gpu_name = "CPU"; vram_total_gb = 0.0
print(f"Device: {device} | GPU: {gpu_name} | VRAM={vram_total_gb:.1f} GB | Torch {torch.__version__} | Py {platform.python_version()}")

RESULT_BASENAME = "bloom_pruning"

Device: cuda | GPU: NVIDIA A100-SXM4-80GB | VRAM=79.3 GB | Torch 2.8.0+cu126 | Py 3.12.11


In [ ]:
# ---------- CodeCarbon-Helfer ----------
def _cleanup_cc_locks():
    for p in [
        "/tmp/.codecarbon.lock",
        _os.path.expanduser("~/.codecarbon/codecarbon.lock"),
        "/content/.codecarbon/codecarbon.lock",
    ]:
        try:
            if _os.path.exists(p):
                _os.remove(p)
                print(f"[CodeCarbon] Lock entfernt: {p}")
        except Exception:
            pass

_os.environ["CODECARBON_CACHE_DIR"] = f"/content/.codecarbon_cache_prune_{int(time.time())}"

def _cc_supported_kwargs():
    base = dict(log_level="error", output_dir=".", measure_power_secs=1, tracking_mode="process")
    try:
        params = inspect.signature(EmissionsTracker.__init__).parameters
        if "allow_multiple_runs" in params: base["allow_multiple_runs"] = True
        if "cloud_provider" in params:      base["cloud_provider"] = "google"
        if "cloud_region" in params:        base["cloud_region"]   = "europe-west10"
        if "country_iso_code" in params:    base["country_iso_code"] = "DEU"
    except Exception:
        pass
    return base

def make_trk(name, out):
    _cleanup_cc_locks()
    return EmissionsTracker(project_name=name, output_file=out, **_cc_supported_kwargs())

def start(tr):
    try:
        tr.start(); return True
    except Exception:
        _cleanup_cc_locks()
        try:
            tr.start(); return True
        except Exception:
            return False

def stop(tr, st):
    if not st:
        return SimpleNamespace(energy_consumed=0.0, emissions=0.0)
    try:
        return tr.stop()
    except Exception:
        return SimpleNamespace(energy_consumed=0.0, emissions=0.0)

def unpack(em):
    if hasattr(em, "energy_consumed") and hasattr(em, "emissions"):
        try: return float(em.energy_consumed), float(em.emissions)
        except: return 0.0, 0.0
    if isinstance(em, dict):
        e = em.get("energy_consumed", 0.0)
        c = em.get("emissions", em.get("emissions_kg", 0.0))
        try: return float(e), float(c)
        except: return 0.0, 0.0
    try: return 0.0, float(em)
    except: return 0.0, 0.0

def read_energy(path):
    try:
        if not _os.path.exists(path): return 0.0
        df = pd.read_csv(path)
        for c in ["energy_consumed","energy_consumed_kwh","energy_consumed (kWh)","energy (kWh)"]:
            if c in df.columns: return float(df[c].iloc[-1])
        for c in df.columns:
            if "energy" in c.lower() and "kwh" in c.lower():
                return float(df[c].iloc[-1])
    except Exception:
        pass
    return 0.0

def measure(phase, fn, prefix):
    logfile = _os.path.join(project_dir, f"{prefix}_{phase}.csv")
    tr = make_trk(f"{prefix}_{phase}", logfile)
    import time as _t
    st = start(tr); t0 = _t.time(); res = fn(); t1 = _t.time()
    em = stop(tr, st); ekwh, co2 = unpack(em)
    if ekwh == 0.0:
        ek = read_energy(logfile)
        if ek: ekwh = ek
    return {"phase": phase, "time_s": t1-t0, "energy_kwh": ekwh, "co2_kg": co2}, res

In [ ]:
# ---------- Eval-Config ----------
MODELS = [
    ("bigscience/bloom-560m", "bloom560m", "b560"),
    ("bigscience/bloom-3b",   "bloom3b",   "b3b"),
]
EVAL = {
    "max_new_tokens": 32,
    "ppl":  {"name":"wikitext","config":"wikitext-2-raw-v1","split":"test[:1%]"},
    "bleu": {"name":"wmt14","config":"de-en","split":"test[:32]"},
}
PROMPTS = [
    "Schreibe einen kurzen Absatz über nachhaltige KI.",
    "Erkläre in einfachen Worten, was Quantisierung in neuronalen Netzen ist.",
    "Nenne drei Vorteile von Mixture-of-Experts-Modellen."
]
bleu_metric = evaluate.load("sacrebleu")

# Pruning-Parameter (20%)
PRUNE_AMOUNT   = 0.20
PRUNE_TARGETS  = ("query_key_value", "dense_h_to_4h", "dense_4h_to_h")

# Eval-Schutz
MAX_LEN_CAP = 256  # ggf. 192/128 für noch weniger VRAM

def autocast_ctx():
    return torch.autocast(device_type="cuda", dtype=torch.float16) if torch.cuda.is_available() else nullcontext()

def safe_max_len(tok, model, fallback=2048, upper=100000):
    cand = getattr(tok, "model_max_length", None)
    if isinstance(cand, int) and 0 < cand < upper: return cand
    cand = getattr(getattr(model, "config", None), "max_position_embeddings", None)
    if isinstance(cand, int) and 0 < cand < upper: return cand
    return fallback

def capture_memory():
    ram = psutil.Process().memory_info().rss
    valloc = torch.cuda.memory_allocated() if torch.cuda.is_available() else 0
    vres  = torch.cuda.memory_reserved()  if torch.cuda.is_available() else 0
    return ram, valloc, vres

def bytes_to_gb(b): return float(b)/(1024**3)

def get_model_device(model):
    try:
        return next(model.parameters()).device
    except StopIteration:
        return torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ---------- CPU-Loader fürs Pruning ----------
from transformers import AutoTokenizer, AutoModelForCausalLM
def load_for_pruning_cpu(model_id: str):
    tok = AutoTokenizer.from_pretrained(model_id, use_fast=True)
    tok.padding_side = "left"; tok.pad_token = tok.eos_token
    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        device_map={"": "cpu"},   # alles auf CPU
        dtype=torch.float32,
        low_cpu_mem_usage=True,
    )
    model.eval()
    return tok, model

In [ ]:
# ---------- Globales Pruning mit OOM-Fallback (layer-weise) ----------
def global_magnitude_prune(model, amount: float):
    params = []
    for name, module in model.named_modules():
        if isinstance(module, nn.Linear) and any(tag in name for tag in PRUNE_TARGETS):
            params.append((module, "weight"))
    if not params:
        print("[Pruning] Keine passenden Linear-Layer gefunden."); return 0.0
    try:
        with torch.no_grad():
            prune.global_unstructured(params, pruning_method=prune.L1Unstructured, amount=amount)
        fallback_used = False
    except Exception as e:
        print("[Pruning] Global OOM -> layer-weise:", repr(e))
        with torch.no_grad():
            for module, name in params:
                prune.l1_unstructured(module, name, amount=amount)
        fallback_used = True

    total, zeros = 0, 0
    for module, _ in params:
        mask = dict(module.named_buffers()).get("weight_mask", None)
        w    = dict(module.named_parameters()).get("weight", None)
        if mask is not None and w is not None:
            zeros += int((w * mask == 0).sum().item())
            total += w.numel()
        try: prune.remove(module, "weight")
        except: pass
    sparsity = (zeros/total) if total else 0.0
    print(f"[Pruning] Modus: {'layer-weise' if fallback_used else 'global'} | Sparsity≈{sparsity:.3f}")
    return sparsity

In [ ]:
# ---------- Evaluation ----------
def simple_generate(model, tok, prompts, max_new_tokens=32):
    dev = get_model_device(model)
    total_gen_tokens, texts = 0, []
    ml = min(MAX_LEN_CAP, safe_max_len(tok, model))
    for p in prompts:
        enc = tok(p, return_tensors="pt", truncation=True, max_length=ml).to(dev)
        room = ml - enc["input_ids"].shape[1]
        cur_new = max(1, min(max_new_tokens, int(room)))
        with torch.no_grad(), autocast_ctx():
            out = model.generate(**enc, max_new_tokens=cur_new, do_sample=False, pad_token_id=tok.eos_token_id)
        gen_len = int(out.shape[1] - enc["input_ids"].shape[1])
        total_gen_tokens += gen_len
        texts.append(tok.decode(out[0], skip_special_tokens=True))
    return texts, total_gen_tokens

def eval_perplexity(model, tok, ds_cfg):
    dev = get_model_device(model)
    ds = load_dataset(ds_cfg["name"], ds_cfg["config"], split=ds_cfg["split"])
    ml = min(MAX_LEN_CAP, safe_max_len(tok, model))
    losses = []
    with torch.no_grad():
        for t in ds["text"]:
            if not isinstance(t, str) or len(t.strip()) < 4: continue
            enc = tok(t, return_tensors="pt", truncation=True, max_length=ml).to(dev)
            with autocast_ctx(): out = model(enc["input_ids"], labels=enc["input_ids"])
            losses.append(float(out.loss.detach().cpu()))
    return math.exp(np.mean(losses)) if losses else None

def eval_bleu_llm(model, tok, ds_cfg, max_new_tokens=32):
    dev = get_model_device(model)
    ds = load_dataset(ds_cfg["name"], ds_cfg["config"], split=ds_cfg["split"])
    ml = min(MAX_LEN_CAP, safe_max_len(tok, model))
    preds, refs = [], []
    with torch.no_grad():
        for ex in ds:
            de, en = ex["translation"]["de"], ex["translation"]["en"]
            prompt = f"Translate to English:\nGerman: {de}\nEnglish:"
            enc = tok(prompt, return_tensors="pt", truncation=True, max_length=ml).to(dev)
            room = ml - enc["input_ids"].shape[1]
            cur_new = max(1, min(max_new_tokens, int(room)))
            with autocast_ctx():
                out = model.generate(**enc, max_new_tokens=cur_new, do_sample=False, pad_token_id=tok.eos_token_id)
            gen = tok.decode(out[0], skip_special_tokens=True)
            hyp = gen.split("English:")[-1].strip().split("\n")[0].strip() or gen.strip()
            preds.append(hyp); refs.append([en])
    return float(bleu_metric.compute(predictions=preds, references=refs)["score"])

In [ ]:
# ---------- Run pro Modell ----------
from transformers import BitsAndBytesConfig

def run_once(model_id: str, alias: str):
    print(f"\n### Starte Pruning: {alias} ({model_id})")

    # 1) CPU laden & PRUNE (kein GPU-VRAM)
    tok, model_cpu = load_for_pruning_cpu(model_id)

    def _do_prune():
        with torch.no_grad():
            sp = global_magnitude_prune(model_cpu, PRUNE_AMOUNT)  # 20%
        return sp
    m_prune, sparsity = measure("prune", _do_prune, f"{RESULT_BASENAME}_{alias}")

    # Geprunte Gewichte speichern & CPU-Modell freigeben
    tmp_dir = os.path.join(project_dir, f"{RESULT_BASENAME}_pruned_ckpt_{alias}")
    model_cpu.save_pretrained(tmp_dir)
    del model_cpu; gc.collect()

    # 2) Für EVAL in 8-bit + CPU-Offload neu laden (GPU-sparend)
    bnb8 = BitsAndBytesConfig(load_in_8bit=True, llm_int8_enable_fp32_cpu_offload=True)
    model = AutoModelForCausalLM.from_pretrained(
        tmp_dir, device_map="auto", quantization_config=bnb8, low_cpu_mem_usage=True,
    )
    model.eval()

    # 3) GEN
    m_gen, (samples, n_tok) = measure("gen",
        lambda: simple_generate(model, tok, PROMPTS, EVAL["max_new_tokens"]),
        f"{RESULT_BASENAME}_{alias}"
    )
    if torch.cuda.is_available(): torch.cuda.empty_cache(); gc.collect()

    # 4) PPL
    m_ppl, ppl = measure("ppl",
        lambda: eval_perplexity(model, tok, EVAL["ppl"]),
        f"{RESULT_BASENAME}_{alias}"
    )
    if torch.cuda.is_available(): torch.cuda.empty_cache(); gc.collect()

    # 5) BLEU
    m_bleu, bleu = measure("bleu",
        lambda: eval_bleu_llm(model, tok, EVAL["bleu"], EVAL["max_new_tokens"]),
        f"{RESULT_BASENAME}_{alias}"
    )
    if torch.cuda.is_available(): torch.cuda.empty_cache(); gc.collect()

    # Gesamt
    total_time   = m_prune["time_s"] + m_gen["time_s"] + m_ppl["time_s"] + m_bleu["time_s"]
    total_energy = m_prune["energy_kwh"] + m_gen["energy_kwh"] + m_ppl["energy_kwh"] + m_bleu["energy_kwh"]
    total_co2    = m_prune["co2_kg"]     + m_gen["co2_kg"]     + m_ppl["co2_kg"]     + m_bleu["co2_kg"]

    ram, valloc, vres = capture_memory()

    per_phase = [m_prune, m_gen, m_ppl, m_bleu]
    for p in per_phase:
        p["alias"] = alias
        p["model_id"] = model_id

    row = dict(
        model_id=model_id, alias=alias, precision="8bit_eval_after_cpu_prune",
        sparsity=round(float(sparsity), 4),
        time_s=total_time, energy_kwh=total_energy, co2_kg=total_co2,
        kg_per_kwh=(total_co2/total_energy) if total_energy else None,
        tokens_out=int(n_tok), ppl=ppl, bleu=bleu,
        ram_GB=bytes_to_gb(ram), vram_alloc_GB=bytes_to_gb(valloc), vram_reserved_GB=bytes_to_gb(vres),
        notes=f"Prune@CPU; Eval 8-bit offload | GPU={gpu_name}, VRAM={vram_total_gb:.1f} GB | prune={PRUNE_AMOUNT:.2f} | MAX_LEN_CAP={MAX_LEN_CAP}"
    )

    # Samples-Datei
    samples_path = os.path.join(project_dir, f"{RESULT_BASENAME}_samples_{alias}.txt")
    with open(samples_path, "w", encoding="utf-8") as f:
        for i, txt in enumerate(samples, 1):
            f.write(f"--- Beispiel {i} ({alias}) ---\n{txt}\n\n")
    print("Beispiele gespeichert:", samples_path)

    return row, per_phase

In [ ]:
# ---------- Ausführen für beide Modelle & Speichern ----------
all_rows, all_phases = [], []
for mid, long_name, alias in MODELS:
    row, phases = run_once(mid, alias)
    all_rows.append(row)
    all_phases.extend(phases)

df  = pd.DataFrame(all_rows).sort_values("alias").reset_index(drop=True)
dfp = pd.DataFrame(all_phases)
dfp["wh_total"] = dfp["energy_kwh"] * 1000.0

out_csv = os.path.join(project_dir, f"{RESULT_BASENAME}_results.csv")
df.to_csv(out_csv, index=False)

out_phase_csv = os.path.join(project_dir, f"{RESULT_BASENAME}_per_phase.csv")
dfp.to_csv(out_phase_csv, index=False)

print("\nErgebnisse (gesamt):")
print(df)
print("Gespeichert (gesamt):", out_csv)

print("\nPer-Phase Übersicht:")
print(dfp[["alias","phase","time_s","energy_kwh","co2_kg"]])
print("Gespeichert (per Phase):", out_phase_csv)


### Starte Pruning: b560 (bigscience/bloom-560m)


tokenizer_config.json:   0%|          | 0.00/222 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/693 [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/693 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/6.01G [00:00<?, ?B/s]

[Pruning] Modus: global | Sparsity≈0.000


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Beispiele gespeichert: /content/drive/MyDrive/LLM-Effizienz/4_3_Effizienzstrategien/bloom_pruning/bloom_pruning_samples_b3b.txt

Ergebnisse (gesamt):
                model_id alias                  precision  sparsity  \
0    bigscience/bloom-3b   b3b  8bit_eval_after_cpu_prune       0.0   
1  bigscience/bloom-560m  b560  8bit_eval_after_cpu_prune       0.0   

       time_s  energy_kwh    co2_kg  kg_per_kwh  tokens_out         ppl  \
0  262.666237    0.009168  0.002454    0.267622          96  234.286083   
1  124.704923    0.003772  0.001009    0.267622          96  327.963333   

        bleu    ram_GB  vram_alloc_GB  vram_reserved_GB  \
0  10.244657  3.723312       3.406985          5.693359   
1   2.708001  2.647381       0.769470          1.029297   

                                               notes  
0  Prune@CPU; Eval 8-bit offload | GPU=NVIDIA A10...  
1  Prune@CPU; Eval 8-bit offload | GPU=NVIDIA A10...  
Gespeichert (gesamt): /content/drive/MyDrive/LLM-Effizienz/4_3_Effi